In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import glob
from skimage.io import imread
import numpy as np
import pandas as pd
import h5py

In [3]:
f = h5py.File('/content/drive/MyDrive/teeth/teeth-300-BN55.h5','r')
def data_gan_fn(long1, batch_size ,type1):
    if type1==1:
      a='train-img'
      b='train-lab'
    else :
      a='vail-img'
      b='vail-lab'
    j = 0
    img_set1=[]
    Lab=[]
    while True:
        img_set1=[]
        Lab=[]
        batch_start = j*batch_size
        if ( batch_start + batch_size ) > long1:
            batch_start2 = 0
            batch_end = long1
            batch_end2 = batch_start + batch_size - long1
            img = f[a][ batch_start : batch_end ] + f[a][ batch_start2 : batch_end2 ]
            lab = f[b][ batch_start : batch_end ] + f[b][ batch_start2 : batch_end2 ]
            for i in range(len(img)):
                img_set1.append(img[i])
                Lab.append(lab[i])
            img_set1=np.stack(img_set1)
            Lab=np.stack(Lab)
            yield img_set1,Lab
        else :
            img = f[a][ batch_start : batch_start + batch_size ]
            lab = f[b][ batch_start : batch_start + batch_size ]
            for i in range(len(img)):
                img_set1.append(img[i])
                Lab.append(lab[i])
            img_set1=np.stack(img_set1)
            Lab=np.stack(Lab)
            yield img_set1,Lab
        j = j + 1
        if (  j*batch_size )  >=  long1:
            j = 0
            img_set1=[]
            Lab=[]

In [4]:
# long1=1752
# batch_size=2
# type1=2
# data_gen  = data_gan_fn(long1, batch_size ,type1)
# next(data_gen)

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,\
MaxPooling2D,Input,Concatenate,BatchNormalization,LeakyReLU

In [6]:
!pip install -U git+https://github.com/qubvel/efficientnet
import efficientnet.keras as efn 
model = efn.EfficientNetB3(weights='imagenet',include_top=False,input_shape=(300,300,3))
model.summary()

  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-yqa024gv
  Running command git clone -q https://github.com/qubvel/efficientnet /tmp/pip-req-build-yqa024gv
     |████████████████████████████████| 51kB 5.9MB/s 
  Created wheel for efficientnet: filename=efficientnet-1.1.1-cp36-none-any.whl size=18422 sha256=2b0ee8aafaef75052e1752bf9d6c3c48d321ac84903f9d58dfc6b46278d2b27a
  Stored in directory: /tmp/pip-ephem-wheel-cache-piua0ck5/wheels/64/60/2e/30ebaa76ed1626e86bfb0cc0579b737fdb7d9ff8cb9522663a
Successfully built efficientnet
44113920/44107200 [==============================] - 0s 0us/step
Model: "efficientnet-b3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 300, 3) 0                                            
____________________________________________________________

In [7]:
model._layers.pop()
model._layers.pop()
model._layers.pop()
model.outputs = model.layers[-1].output
# 加層
x=Flatten()(model.outputs)
x=Dropout(0.2)(x)
x=Dense(512, activation='relu',kernel_initializer='he_normal')(x)
x=Dropout(0.2)(x)
x=Dense(256, activation='relu',kernel_initializer='he_normal')(x)
x=Dropout(0.2)(x)
x=Dense(2, activation='softmax',kernel_initializer='he_normal')(x)
# # 重新建立模型結構
model=Model(model.input,x)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 150, 150, 40) 1080        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 150, 150, 40) 160         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 150, 150, 40) 0           stem_bn[0][0]                    
______________________________________________________________________________________________

In [8]:
import glob
from skimage.io import imread,imsave,imshow
from tensorflow.keras.optimizers import Adam
from skimage.transform import resize
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
import random
from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping , CSVLogger ,ReduceLROnPlateau
import tensorflow

In [9]:
model.compile( optimizer = Adam( lr = 1e-4 ) , 
              loss = 'categorical_crossentropy' , 
              metrics=['accuracy'] )

In [10]:
epoch = 500
batch_size = 5

In [ ]:
data_gen  = data_gan_fn(1752, batch_size ,1)
val_data = data_gan_fn(456 ,batch_size ,2)

steps_per_epoch = int( np.ceil( 1756/batch_size ) )
validation_steps = int( np.ceil( 456/batch_size ) )
model_path =   '/content/drive/MyDrive/teeth/saved_models/300-t3.h5' 
check_point = ModelCheckpoint(model_path, monitor='val_loss' , 
                              save_best_only= True , verbose = 1)
csv_logger = CSVLogger( '/content/drive/MyDrive/teeth/logs/300-t3.log ' )
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, 
#                     patience=50, min_delta=0.0001)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                            patience=50, min_lr=0.000001,verbose=1)
result = model.fit_generator( data_gen , epochs=epoch , verbose=True , 
                              steps_per_epoch=steps_per_epoch ,
                              callbacks=[reduce_lr,check_point,csv_logger],
                              validation_data = val_data,
                              validation_steps = validation_steps)



/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/500
